In [1]:
# Cloning YOLO github
!git clone https://github.com/ultralytics/yolov5
!pip install -r yolov5/requirements.txt
!pip install utils
!pip install pylabel > /dev/null
!pip install clearml


Cloning into 'yolov5'...
remote: Enumerating objects: 15705, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 15705 (delta 9), reused 15 (delta 3), pack-reused 15672
Receiving objects: 100% (15705/15705), 14.51 MiB | 24.11 MiB/s, done.
Resolving deltas: 100% (10754/10754), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.4/591.4 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 20.8 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 59.8.0
    Uninstalling setuptools-59.8.0:
      Successfully uninstalled setuptools-59.8.0
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.5.4.60
    Uninstalling opencv-python-4.5.4.60:
      Successfully uninsta

# Importing the Necessary Libraries

In [2]:
import torch
import os 
import random
import shutil
import xml.etree.ElementTree as ET
import numpy as np
import matplotlib.pyplot as plt
import logging
import os 
import zipfile
import yaml
import PIL

from IPython.display import Image  # for displaying images
from sklearn.model_selection import train_test_split
from pylabel import importer
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import os
import torch
import torch.nn as nn
import torch.optim as optim



logging.getLogger().setLevel(logging.CRITICAL)
random.seed(42)


# Convert COCO JSON Annotations to YOLO TXT Files

## Import COCO Annotations

In [3]:
# Copy images_raw to working directory
# Note: This may take some time depending on the size of your images_raw folder
!cp -r /kaggle/input/dataset-yolo ./


Move labels folder inside the yolov5 directory

# Adjusting the dataset.yaml file

* `path`: images
* `train`: train
* `val`: val
* `test`: test

In [4]:
# Viewing the original unprocessed yaml file

yaml_params = {}
with open(r'/kaggle/working/dataset-yolo/Cigarrette_Butts/data.yaml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    yaml_file_list = yaml.load(file, Loader=yaml.FullLoader)
    yaml_params = yaml_file_list
    print(yaml_file_list)

{'train': 'Cigarrette_Butts/train/images', 'val': 'Cigarrette_Butts/val/images', 'test': 'Cigarrette_Butts/test/images', 'nc': 4, 'names': {0: 'CigaretteButt', 1: 'Garbage Bag', 2: 'Paper Bag', 3: 'Plastic Bag'}}


In [5]:
# Adjusting the parameters of the yaml file
yaml_params['train'] = '/kaggle/working/dataset-yolo/Cigarrette_Butts/train/images'
yaml_params['val'] = '/kaggle/working/dataset-yolo/Cigarrette_Butts/val/images'
yaml_params['test'] = '/kaggle/working/dataset-yolo/Cigarrette_Butts/test/images'
yaml_params['names'][1] = 'Paper Bag'
yaml_params['names'][2] = 'Garbage Bag'
yaml_params['names'][3] = 'Plastic Bag'

yaml_params

{'train': '/kaggle/working/dataset-yolo/Cigarrette_Butts/train/images',
 'val': '/kaggle/working/dataset-yolo/Cigarrette_Butts/val/images',
 'test': '/kaggle/working/dataset-yolo/Cigarrette_Butts/test/images',
 'nc': 4,
 'names': {0: 'CigaretteButt',
  1: 'Paper Bag',
  2: 'Garbage Bag',
  3: 'Plastic Bag'}}

In [6]:
# Overwriting the new params from the previous ones.
with open(r'dataset.yaml', 'w') as file:
    documents = yaml.dump(yaml_params, file)

# Training the Custom Dataset using YOLOv5

Now, we train the network. We use various flags to set options regarding training.

## Training using YOLOv5

In [7]:
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=5EEUPBPWIWBGVZDW265X
%env CLEARML_API_SECRET_KEY=G27YQwb8xSAGMiiOFJLAmBszaUAs1Tte3Pq5ty9TTFl1YsWsaw

from clearml import Task
task = Task.init(project_name="YoloFinetune", task_name="Kaggle")

env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=5EEUPBPWIWBGVZDW265X
env: CLEARML_API_SECRET_KEY=G27YQwb8xSAGMiiOFJLAmBszaUAs1Tte3Pq5ty9TTFl1YsWsaw
ClearML Task: created new task id=033f045cf0f4481bbc35c2201389f947
2023-05-28 02:51:21,001 - clearml.Repository Detection - WARNING - Failed accessing the jupyter server(s): []
2023-05-28 02:51:21,028 - clearml.Task - INFO - No repository found, storing script code instead
ClearML results page: https://app.clear.ml/projects/8425d1a9dc0f43a182e8175c34715835/experiments/033f045cf0f4481bbc35c2201389f947/output/log


In [8]:
# with open("/kaggle/working/yolov5/runs/train/yolo_finetune2/weights/best.pt", "rb") as source_file:
#     weights = source_file.read()

# with open("best.pt", "wb") as destination_file:
#     destination_file.write(weights)


In [9]:
# def fitness(x): 
#     # Model fitness as a weighted combination of metrics 
#     w = [0.0, 0.0, 0.1, 0.9]  # weights for [P, R, mAP@0.5, mAP@0.5:0.95] 
#     return (x[:, :4] * w).sum(1) 
# !python /kaggle/working/yolov5/train.py --img 640 --cfg /kaggle/working/yolov5/models/yolov5s.yaml --hyp /kaggle/working/yolov5/data/hyps/hyp.scratch-low.yaml --batch 64 --epochs 300 --data /kaggle/working/dataset.yaml --weights "yolov5s.pt" --workers 100 --name yolo_finetune

<a href="/kaggle/working/yolov5/runs/train/yolo_finetune2/weights/best.pt">Download File</a>


# Predict on our own Test Image